In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import os

import march_madness as mm
import random

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures

In [3]:
analysis = mm.Analysis()

In [4]:
analysis.load_training_data('MDataFiles_Stage2/calculated features 1985-2021.csv')

In [5]:
analysis.load_validation_data('MDataFiles_Stage2/calculated features 2022.csv')

# Overview

This notebook tests all the models developed for phase 1 of my model devlopepment against the actional results of the 2022 Tournament.

The detailed process of how I developed these models can be found here. TODO: insert hyperlink

# Notable Results

* The predictions based solely on the historical win percentages of each seed-to-seed match-up performed remarkably well with a competition score of 0.60497. That score would've placed 52nd out 930 teams.
* The Logistic Regression model appeared to generalize best with fewer features. Its performance got worse as more features were added its best result was actually using the pruned feature set. The score of 0.61613 using the pruned feature set would've been good for 143rd.
* The Random Forest model (max_features controlled) performed batter as more features were added. This model performed worse on the training data but actually performed better in the 2022 data. Using all features, this model scored 0.63989 for 342nd.
* The Random Forest models that we controlled using 'max_depth' performed better across the board. The final model using all the features scored 0.60753 which would've placed 70th.

# Baseline Cases

## 50/50 Predictions

In [6]:
feature_keys = []

In [7]:
X_train, y_train = analysis.extract_training_data(feature_keys=feature_keys)

In [8]:
X_test, y_test = analysis.extract_validation_data(feature_keys=feature_keys)

In [9]:
naive_predictions = np.ones(y_test.shape) * 0.5

In [10]:
analysis.score_model_predictions(y_test,naive_predictions)

0.6931471805599454

## Seed Win Percentages

In [11]:
feature_keys = ['tourney seed']

In [12]:
X_train, y_train = analysis.extract_training_data(feature_keys=feature_keys)

In [13]:
X_test, y_test = analysis.extract_validation_data(feature_keys=feature_keys)

In [14]:
seed_predictions = analysis.generate_seed_win_predictions(X_test)

In [15]:
analysis.score_model_predictions(y_test,seed_predictions)

0.6049654680173446

## Seed Only (As Continuous Value)

In [16]:
feature_keys = ['tourney seed']

In [17]:
X_train, y_train = analysis.extract_training_data(feature_keys=feature_keys)

In [18]:
X_test, y_test = analysis.extract_validation_data(feature_keys=feature_keys)

In [23]:
logreg = LogisticRegression().fit(X_train,y_train)

In [24]:
logreg.score(X_train,y_train)

0.7129909365558912

In [25]:
logreg.score(X_test,y_test)

0.6567164179104478

In [26]:
logreg_pred = mm.bound_predictions(logreg.predict_proba(X_test)[:,1])

In [27]:
analysis.score_model_predictions(y_test,logreg_pred)

0.6257235531430584

# Model Development

## RPI (No Seed)

In [42]:
feature_keys = ['weighted win pct','owp','oowp']

In [43]:
X_train, y_train = analysis.extract_training_data(feature_keys=feature_keys)

In [44]:
X_test, y_test = analysis.extract_validation_data(feature_keys=feature_keys)

In [49]:
logreg = LogisticRegression().fit(X_train,y_train)

In [50]:
logreg.score(X_train,y_train)

0.7060854553301683

In [51]:
logreg.score(X_test,y_test)

0.6417910447761194

In [52]:
forest = RandomForestClassifier(n_estimators=100,max_features=3,random_state=100).fit(X_train,y_train)

In [53]:
forest.score(X_train,y_train)

1.0

In [54]:
forest.score(X_test,y_test)

0.6119402985074627

In [55]:
logreg_pred = mm.bound_predictions(logreg.predict_proba(X_test)[:,1])

In [56]:
forest_pred = mm.bound_predictions(forest.predict_proba(X_test)[:,1])

In [57]:
analysis.score_model_predictions(y_test,logreg_pred)

0.6203345944883191

In [58]:
analysis.score_model_predictions(y_test,forest_pred)

0.6669379903082573

## RPI + Seed

In [59]:
feature_keys = ['tourney seed','weighted win pct','owp','oowp']

In [60]:
X_train, y_train = analysis.extract_training_data(feature_keys=feature_keys)

In [61]:
X_test, y_test = analysis.extract_validation_data(feature_keys=feature_keys)

In [66]:
logreg = LogisticRegression().fit(X_train,y_train)

In [67]:
logreg.score(X_train,y_train)

0.7069486404833837

In [68]:
logreg.score(X_test,y_test)

0.6567164179104478

In [69]:
forest = RandomForestClassifier(n_estimators=100,max_features=3,random_state=100).fit(X_train,y_train)

In [70]:
forest.score(X_train,y_train)

1.0

In [71]:
forest.score(X_test,y_test)

0.582089552238806

In [72]:
logreg_pred = mm.bound_predictions(logreg.predict_proba(X_test)[:,1])

In [73]:
forest_pred = mm.bound_predictions(forest.predict_proba(X_test)[:,1])

In [74]:
analysis.score_model_predictions(y_test,logreg_pred)

0.6324629611748651

In [75]:
analysis.score_model_predictions(y_test,forest_pred)

0.6723064521856913

## Margin of Victory

In [76]:
feature_keys = ['tourney seed','weighted win pct','owp','oowp','avg win margin','std win margin','avg loss margin',
                'std loss margin','capped avg win margin','capped std win margin','capped avg loss margin',
                'capped std loss margin','close wins','close losses']

In [77]:
X_train, y_train = analysis.extract_training_data(feature_keys=feature_keys)

In [78]:
X_test, y_test = analysis.extract_validation_data(feature_keys=feature_keys)

In [83]:
logreg = LogisticRegression().fit(X_train,y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [84]:
logreg.score(X_train,y_train)

0.7112645662494606

In [85]:
logreg.score(X_test,y_test)

0.6417910447761194

In [86]:
forest = RandomForestClassifier(n_estimators=100,max_features=3,random_state=100).fit(X_train,y_train)

In [87]:
forest.score(X_train,y_train)

1.0

In [88]:
forest.score(X_test,y_test)

0.7164179104477612

In [89]:
logreg_pred = mm.bound_predictions(logreg.predict_proba(X_test)[:,1])

In [90]:
forest_pred = mm.bound_predictions(forest.predict_proba(X_test)[:,1])

In [91]:
analysis.score_model_predictions(y_test,logreg_pred)

0.679424234850981

In [92]:
analysis.score_model_predictions(y_test,forest_pred)

0.6346503346666067

## Quality Wins

In [94]:
feature_keys = ['tourney seed','weighted win pct','owp','oowp','avg win margin','std win margin','avg loss margin',
                'std loss margin','capped avg win margin','capped std win margin','capped avg loss margin',
                'capped std loss margin','close wins','close losses','weighted top64 wins','weighted top32 wins',
                'weighted top16 wins','weighted top8 wins','weighted top64 losses','weighted top32 losses',
                'weighted top16 losses','weighted top8 losses']

In [95]:
X_train, y_train = analysis.extract_training_data(feature_keys=feature_keys)

In [96]:
X_test, y_test = analysis.extract_validation_data(feature_keys=feature_keys)

In [101]:
logreg = LogisticRegression().fit(X_train,y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [102]:
logreg.score(X_train,y_train)

0.7293914544669832

In [103]:
logreg.score(X_test,y_test)

0.6417910447761194

In [104]:
forest = RandomForestClassifier(n_estimators=1000,max_features=3,random_state=100).fit(X_train,y_train)

In [105]:
forest.score(X_train,y_train)

1.0

In [106]:
forest.score(X_test,y_test)

0.6268656716417911

In [107]:
logreg_pred = mm.bound_predictions(logreg.predict_proba(X_test)[:,1])

In [108]:
forest_pred = mm.bound_predictions(forest.predict_proba(X_test)[:,1])

In [109]:
analysis.score_model_predictions(y_test,logreg_pred)

0.6897650925349336

In [110]:
analysis.score_model_predictions(y_test,forest_pred)

0.6478773817698826

## Late Season Form

In [111]:
feature_keys = ['tourney seed','weighted win pct','owp','oowp','avg win margin','std win margin','avg loss margin',
                'std loss margin','capped avg win margin','capped std win margin','capped avg loss margin',
                'capped std loss margin','close wins','close losses','weighted top64 wins','weighted top32 wins',
                'weighted top16 wins','weighted top8 wins','weighted top64 losses','weighted top32 losses',
                'weighted top16 losses','weighted top8 losses','last10 win pct','last10 weighted win pct',
                'last5 win pct','last5 weighted win pct','conference tourney wins','conference champ']

In [112]:
X_train, y_train = analysis.extract_training_data(feature_keys=feature_keys)

In [113]:
X_test, y_test = analysis.extract_validation_data(feature_keys=feature_keys)

In [118]:
logreg = LogisticRegression(max_iter=1000).fit(X_train,y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [119]:
logreg.score(X_train,y_train)

0.7319810099266293

In [120]:
logreg.score(X_test,y_test)

0.6567164179104478

In [121]:
forest = RandomForestClassifier(n_estimators=1000,max_features=3,random_state=100).fit(X_train,y_train)

In [122]:
forest.score(X_train,y_train)

1.0

In [123]:
forest.score(X_test,y_test)

0.6268656716417911

In [124]:
logreg_pred = mm.bound_predictions(logreg.predict_proba(X_test)[:,1])

In [125]:
forest_pred = mm.bound_predictions(forest.predict_proba(X_test)[:,1])

In [126]:
analysis.score_model_predictions(y_test,logreg_pred)

0.676961494541948

In [127]:
analysis.score_model_predictions(y_test,forest_pred)

0.6398966806324071

## Pruned Feature Set

In [131]:
feature_keys = ['tourney seed','weighted win pct','owp','oowp','capped avg win margin','capped std win margin',
                'capped avg loss margin','capped std loss margin','close wins','close losses','weighted top64 wins',
                'weighted top16 wins','weighted top64 losses','weighted top16 losses','last10 weighted win pct',
                'last5 weighted win pct','conference tourney wins','conference champ']

In [132]:
X_train, y_train = analysis.extract_training_data(feature_keys=feature_keys)

In [133]:
X_test, y_test = analysis.extract_validation_data(feature_keys=feature_keys)

In [138]:
logreg = LogisticRegression().fit(X_train,y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [139]:
logreg.score(X_train,y_train)

0.7229175658178679

In [140]:
logreg.score(X_test,y_test)

0.6716417910447762

In [141]:
max_features = round(np.sqrt(len(feature_keys)))
max_features

4

In [142]:
forest = RandomForestClassifier(n_estimators=1000,max_features=3,random_state=100).fit(X_train,y_train)

In [143]:
forest.score(X_train,y_train)

1.0

In [144]:
forest.score(X_test,y_test)

0.6567164179104478

In [145]:
logreg_pred = mm.bound_predictions(logreg.predict_proba(X_test)[:,1])

In [146]:
forest_pred = mm.bound_predictions(forest.predict_proba(X_test)[:,1])

In [147]:
analysis.score_model_predictions(y_test,logreg_pred)

0.6161292833078834

In [148]:
analysis.score_model_predictions(y_test,forest_pred)

0.6197241436197702

# Random Forest - Depth Limit Models

## RPI (No Seed)

In [158]:
feature_keys = ['weighted win pct','owp','oowp']

In [159]:
X_train, y_train = analysis.extract_training_data(feature_keys=feature_keys)

In [160]:
X_test, y_test = analysis.extract_validation_data(feature_keys=feature_keys)

In [165]:
forest = RandomForestClassifier(n_estimators=1000,max_features='sqrt',max_depth=5,random_state=100).fit(X_train,y_train)

In [166]:
forest.score(X_train,y_train)

0.736296935692706

In [167]:
forest.score(X_test,y_test)

0.6268656716417911

In [168]:
forest_pred = mm.bound_predictions(forest.predict_proba(X_test)[:,1])

In [169]:
analysis.score_model_predictions(y_test,forest_pred)

0.588838222402661

## RPI + Seed

In [170]:
feature_keys = ['tourney seed','weighted win pct','owp','oowp']

In [171]:
X_train, y_train = analysis.extract_training_data(feature_keys=feature_keys)

In [172]:
X_test, y_test = analysis.extract_validation_data(feature_keys=feature_keys)

In [177]:
forest = RandomForestClassifier(n_estimators=1000,max_features='sqrt',max_depth=5,random_state=100).fit(X_train,y_train)

In [178]:
forest.score(X_train,y_train)

0.7457919723780752

In [179]:
forest.score(X_test,y_test)

0.6716417910447762

In [180]:
forest_pred = mm.bound_predictions(forest.predict_proba(X_test)[:,1])

In [181]:
analysis.score_model_predictions(y_test,forest_pred)

0.5925742244796036

## Margin of Victory

In [182]:
feature_keys = ['tourney seed','weighted win pct','owp','oowp','avg win margin','std win margin','avg loss margin',
                'std loss margin','capped avg win margin','capped std win margin','capped avg loss margin',
                'capped std loss margin','close wins','close losses']

In [183]:
X_train, y_train = analysis.extract_training_data(feature_keys=feature_keys)

In [184]:
X_test, y_test = analysis.extract_validation_data(feature_keys=feature_keys)

In [189]:
forest = RandomForestClassifier(n_estimators=1000,max_features='sqrt',max_depth=5,random_state=100).fit(X_train,y_train)

In [190]:
forest.score(X_train,y_train)

0.7643504531722054

In [191]:
forest.score(X_test,y_test)

0.6417910447761194

In [192]:
forest_pred = mm.bound_predictions(forest.predict_proba(X_test)[:,1])

In [193]:
analysis.score_model_predictions(y_test,forest_pred)

0.6152380299843667

## Quality Wins

In [194]:
feature_keys = ['tourney seed','weighted win pct','owp','oowp','avg win margin','std win margin','avg loss margin',
                'std loss margin','capped avg win margin','capped std win margin','capped avg loss margin',
                'capped std loss margin','close wins','close losses','weighted top64 wins','weighted top32 wins',
                'weighted top16 wins','weighted top8 wins','weighted top64 losses','weighted top32 losses',
                'weighted top16 losses','weighted top8 losses']

In [195]:
X_train, y_train = analysis.extract_training_data(feature_keys=feature_keys)

In [196]:
X_test, y_test = analysis.extract_validation_data(feature_keys=feature_keys)

In [201]:
forest = RandomForestClassifier(n_estimators=1000,max_features='sqrt',max_depth=5,random_state=100).fit(X_train,y_train)

In [202]:
forest.score(X_train,y_train)

0.7686663789382823

In [203]:
forest.score(X_test,y_test)

0.6417910447761194

In [204]:
forest_pred = mm.bound_predictions(forest.predict_proba(X_test)[:,1])

In [205]:
analysis.score_model_predictions(y_test,forest_pred)

0.6119044517668185

## Late Season Form

In [206]:
feature_keys = ['tourney seed','weighted win pct','owp','oowp','avg win margin','std win margin','avg loss margin',
                'std loss margin','capped avg win margin','capped std win margin','capped avg loss margin',
                'capped std loss margin','close wins','close losses','weighted top64 wins','weighted top32 wins',
                'weighted top16 wins','weighted top8 wins','weighted top64 losses','weighted top32 losses',
                'weighted top16 losses','weighted top8 losses','last10 win pct','last10 weighted win pct',
                'last5 win pct','last5 weighted win pct','conference tourney wins','conference champ']

In [207]:
X_train, y_train = analysis.extract_training_data(feature_keys=feature_keys)

In [208]:
X_test, y_test = analysis.extract_validation_data(feature_keys=feature_keys)

In [213]:
forest = RandomForestClassifier(n_estimators=1000,max_features='sqrt',max_depth=5,random_state=100).fit(X_train,y_train)

In [214]:
forest.score(X_train,y_train)

0.7734138972809668

In [215]:
forest.score(X_test,y_test)

0.6268656716417911

In [216]:
forest_pred = mm.bound_predictions(forest.predict_proba(X_test)[:,1])

In [217]:
analysis.score_model_predictions(y_test,forest_pred)

0.60753019317656